The code to process German Credit data, edit/train models, and perform adversarial debiasing. 

Necessary libraries for the notebook.

In [1]:
import os
import tensorflow as tf
import tf2onnx
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from scipy.io import savemat
import numpy as np
import pandas as pd
import warnings
import csv

2024-07-10 21:41:03.989184: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 21:41:17.502355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Data Preprocessing

In [2]:
# Custom preprocessing function for the German dataset
def german_custom_preprocessing(df):
    def group_credit_hist(x):
        if x in ['A30', 'A31', 'A32']:
            return 'None/Paid'
        elif x == 'A33':
            return 'Delay'
        elif x == 'A34':
            return 'Other'
        else:
            return 'NA'

    def group_employ(x):
        if x == 'A71':
            return 'Unemployed'
        elif x in ['A72', 'A73']:
            return '1-4 years'
        elif x in ['A74', 'A75']:
            return '4+ years'
        else:
            return 'NA'

    def group_savings(x):
        if x in ['A61', 'A62']:
            return '<500'
        elif x in ['A63', 'A64']:
            return '500+'
        elif x == 'A65':
            return 'Unknown/None'
        else:
            return 'NA'

    def group_status(x):
        if x in ['A11', 'A12']:
            return '<200'
        elif x in ['A13']:
            return '200+'
        elif x == 'A14':
            return 'None'
        else:
            return 'NA'

    status_map = {'A91': 1, 'A93': 1, 'A94': 1, 'A92': 0, 'A95': 0}  # 1: 'male'
    df['sex'] = df['personal_status'].replace(status_map)

    df['credit_history'] = df['credit_history'].apply(lambda x: group_credit_hist(x))
    df['savings'] = df['savings'].apply(lambda x: group_savings(x))
    df['employment'] = df['employment'].apply(lambda x: group_employ(x))
    df['status'] = df['status'].apply(lambda x: group_status(x))

    df.credit.replace([1, 2], [1, 0], inplace=True)

    return df

def load_german():
    filepath = '../data/german/german.data'
    column_names = ['status', 'month', 'credit_history', 'purpose', 'credit_amount', 'savings', 'employment',
                    'investment_as_income_percentage', 'personal_status', 'other_debtors', 'residence_since', 
                    'property', 'age', 'installment_plans', 'housing', 'number_of_credits', 'skill_level', 
                    'people_liable_for', 'telephone', 'foreign_worker', 'credit']
    na_values = []
    df = pd.read_csv(filepath, sep=' ', header=None, names=column_names, na_values=na_values)
    
    df = german_custom_preprocessing(df)
    feat_to_drop = ['personal_status']
    df = df.drop(feat_to_drop, axis=1)
    
    # Encode categorical features
    cat_feat = ['status', 'credit_history', 'purpose', 'savings', 'employment', 'other_debtors', 'property', 
                'installment_plans', 'housing', 'skill_level', 'telephone', 'foreign_worker']
    for col in cat_feat:
        df[col] = LabelEncoder().fit_transform(df[col])
    
    # Encode the target variable
    label_name = 'credit'
    
    X = df.drop(labels=[label_name], axis=1, inplace=False)
    y = df[label_name]
    
    # Extract the protected attribute ('sex')
    protected_attribute = X['sex'].values
    
    # Split the data into training and testing sets
    seed = 42
    X_train, X_test, y_train, y_test, protected_train, protected_test = train_test_split(
        X, y, protected_attribute, test_size=0.15, random_state=seed
    )
    
    # One-hot encode the labels
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)
    
    return X_train, X_test, y_train, y_test, protected_train, protected_test

# Saves data for use in verification
def load_and_save_german_data():
    X_train, X_test, y_train, y_test, _, _ = load_german()
    
    # Scaling numerical features with MinMaxScaler
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Prepare data dictionary to save as .mat file
    data_dict = {
        'X': X_test, 
        'y': y_test   
    }
    
    # Save to .mat file for use in MATLAB
    savemat("./processed_data/german_data.mat", data_dict)
    print("Data saved to german_data.mat")

    return X_train, X_test, y_train, y_test

### Model Editing

Method to save the models as onnx files for verification. 

In [3]:
# Function to save the model as ONNX format
def save_model_onnx(model, input_shape, onnx_file_path):
    # Create a dummy input tensor with the correct input shape (batch_size, input_shape)
    dummy_input = tf.random.normal([1] + list(input_shape))

    # Convert the model to ONNX
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model, 
                                                                      input_signature=(tf.TensorSpec(shape=[None] + list(input_shape), dtype=tf.float32),),
                                                                      opset=13)
    
    # Save the ONNX model to the specified path
    with open(onnx_file_path, "wb") as f:
        f.write(model_proto.SerializeToString())
    
    print(f"Model has been saved in ONNX format at {onnx_file_path}")

Change the models so they are able to be used in FairNNV. FairNNV cannot handle sigmoid so shift to softmax and adjust final layers. 

In [4]:
# Function to modify a model for multiclass classification
def modify_model_for_multiclass(model_path, num_classes):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=UserWarning)
        model = load_model(model_path)

    # Create a new input layer with the correct shape
    new_input = tf.keras.layers.Input(shape=(20,))
    x = new_input

    # Transfer the layers except the last one
    for layer in model.layers[:-1]:
        x = layer(x)

    # Create a new output layer
    output = tf.keras.layers.Dense(num_classes, activation='softmax', name='new_output')(x)
    
    # Create a new model
    new_model = tf.keras.models.Model(inputs=new_input, outputs=output)
    
    return new_model

# Ensure the save directories exist
model_dir = './german/german_h5'
save_dir = './german/german_keras'
onnx_save_dir = './german/german_onnx'
num_classes = 2

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(onnx_save_dir):
    os.makedirs(onnx_save_dir)

# Modify each model in the directory to remove sigmoid
for model_file in os.listdir(model_dir):
    if model_file.endswith('.h5'):
        model_path = os.path.join(model_dir, model_file)
        new_model = modify_model_for_multiclass(model_path, num_classes)
        
        # Update the model's loss function
        new_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Save the modified model
        save_path = os.path.join(save_dir, model_file.replace('.h5', '.keras'))
        new_model.save(save_path)


Re-train models. 

In [5]:
# Load and preprocess the German dataset
X_train, X_test, y_train, y_test = load_and_save_german_data()

for model_file in os.listdir(save_dir):
    if model_file.endswith('.keras'):
        model_path = os.path.join(save_dir, model_file)
        
        try:
            # Load the modified model
            print(f"Loading model {model_file}")
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)
                model = load_model(model_path)

            # Reinitialize the optimizer
            model.compile(
                optimizer=Adam(),
                loss='categorical_crossentropy', 
                metrics=['accuracy']
            )

            # Fit the model
            print(f"Training model {model_file}")
            history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

            # Evaluate the model
            y_pred = model.predict(X_test)
            y_pred_classes = np.argmax(y_pred, axis=1)
            accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_classes)

            print(f"Model {model_file} - Accuracy: {accuracy}")

            # Save the retrained model
            model.save(model_path)
            print(f"Model {model_file} retrained and saved successfully.")

            # Save the model as ONNX
            onnx_save_path = os.path.join(onnx_save_dir, model_file.replace('.keras', '.onnx'))
            save_model_onnx(model, (20,), onnx_save_path)

        except Exception as e:
            print(f"Failed to process {model_file}. Error: {e}")

Data saved to german_data.mat
Loading model GC-1.keras
Training model GC-1.keras
Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5173 - loss: 0.7113 - val_accuracy: 0.6941 - val_loss: 0.6186
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7063 - loss: 0.5806 - val_accuracy: 0.6882 - val_loss: 0.6028
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6971 - loss: 0.5836 - val_accuracy: 0.7000 - val_loss: 0.5954
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7073 - loss: 0.5725 - val_accuracy: 0.7000 - val_loss: 0.5888
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7259 - loss: 0.5461 - val_accuracy: 0.6941 - val_loss: 0.5824
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6982 - loss: 0.5655 - val_accuracy: 0.6882 - val_loss: 0.5766
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7022 - loss: 0.5629 - val_accuracy: 0.6882 - val_loss: 0.5746
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━

2024-07-10 21:41:48.073678: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:41:48.073819: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:41:48.091988: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:41:48.092130: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6942 - loss: 1.0844 - val_accuracy: 0.6824 - val_loss: 0.5855
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7127 - loss: 0.5564 - val_accuracy: 0.6882 - val_loss: 0.5745
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7218 - loss: 0.5249 - val_accuracy: 0.6882 - val_loss: 0.5706
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6990 - loss: 0.5621 - val_accuracy: 0.6882 - val_loss: 0.5677
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7233 - loss: 0.5166 - val_accuracy: 0.7059 - val_loss: 0.5654
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7271 - loss: 0.5426 - val_accuracy: 0.7000 - val_loss: 0.5627
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7210 - loss: 0.5300 - val_accuracy: 0.7000 - val_loss: 0.5604
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7458 - loss: 0.5007 - val_accuracy: 0.7000 - val_loss: 0.5606
Epo

2024-07-10 21:41:51.527744: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:41:51.527876: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:41:51.545422: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:41:51.545502: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5901 - loss: 0.7036 - val_accuracy: 0.6529 - val_loss: 0.6845
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6807 - loss: 0.6795 - val_accuracy: 0.6706 - val_loss: 0.6752
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6920 - loss: 0.6578 - val_accuracy: 0.6647 - val_loss: 0.6629
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7093 - loss: 0.6414 - val_accuracy: 0.6765 - val_loss: 0.6531
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7170 - loss: 0.6131 - val_accuracy: 0.6765 - val_loss: 0.6447
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6875 - loss: 0.6210 - val_accuracy: 0.6765 - val_loss: 0.6359
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6955 - loss: 0.6164 - val_accuracy: 0.6765 - val_loss: 0.6292
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6949 - loss: 0.6258 - val_accuracy: 0.6765 - val_loss: 0.6230
Epo

...

2024-07-10 21:41:55.433003: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:41:55.433095: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:41:55.450674: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:41:55.450761: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6520 - loss: 0.6911 - val_accuracy: 0.6941 - val_loss: 0.6854
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7330 - loss: 0.6819 - val_accuracy: 0.6941 - val_loss: 0.6784
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6741 - loss: 0.6785 - val_accuracy: 0.6941 - val_loss: 0.6723
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6932 - loss: 0.6709 - val_accuracy: 0.6941 - val_loss: 0.6663
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6933 - loss: 0.6649 - val_accuracy: 0.6941 - val_loss: 0.6608
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6966 - loss: 0.6590 - val_accuracy: 0.6941 - val_loss: 0.6560
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7038 - loss: 0.6526 - val_accuracy: 0.6941 - val_loss: 0.6515
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7270 - loss: 0.6419 - val_accuracy: 0.6941 - val_loss: 0.6472
Epo

2024-07-10 21:41:59.709202: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:41:59.709322: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:41:59.729225: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:41:59.729341: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6530 - loss: 0.6861 - val_accuracy: 0.6941 - val_loss: 0.6558
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7178 - loss: 0.6389 - val_accuracy: 0.6941 - val_loss: 0.6179
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7075 - loss: 0.6038 - val_accuracy: 0.6941 - val_loss: 0.6130
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6815 - loss: 0.6205 - val_accuracy: 0.6941 - val_loss: 0.6099
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7116 - loss: 0.5865 - val_accuracy: 0.6941 - val_loss: 0.6060
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7018 - loss: 0.5938 - val_accuracy: 0.6941 - val_loss: 0.6014
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7074 - loss: 0.5843 - val_accuracy: 0.6941 - val_loss: 0.5963
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6966 - loss: 0.5891 - val_accuracy: 0.6941 - val_loss: 0.5906
Epo

2024-07-10 21:42:04.178906: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:42:04.178987: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:42:04.211691: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:42:04.211774: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


### Adversairal Debiasing 

In [6]:
def save_metrics_to_csv(filename, model_file, model_name, classification_accuracy, balanced_accuracy, disparate_impact, equal_opportunity_difference, average_odds_difference):
    # Check if the file exists to write the header only once
    file_exists = os.path.isfile(filename)

    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            # Write the header if the file does not exist
            writer.writerow(['Model File', 'Model', 'Classification Accuracy', 'Balanced Accuracy', 'Disparate Impact', 'Equal Opportunity Difference', 'Average Odds Difference'])
        
        # Write the metrics
        writer.writerow([model_file, model_name, classification_accuracy, balanced_accuracy, disparate_impact, equal_opportunity_difference, average_odds_difference])

Various metrics for evaluation including accuracy and fairness.

In [7]:
# Metrics calculation functions
def classification_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def balanced_accuracy(y_true, y_pred):
    classes = np.unique(y_true)
    recall_scores = []
    for cls in classes:
        true_positives = np.sum((y_true == cls) & (y_pred == cls))
        possible_positives = np.sum(y_true == cls)
        recall_scores.append(true_positives / possible_positives)
    return np.mean(recall_scores)

def disparate_impact(y_true, y_pred, protected_attribute):
    privileged = protected_attribute == 1
    unprivileged = protected_attribute == 0
    if np.sum(privileged) == 0 or np.sum(unprivileged) == 0:
        return np.nan
    privileged_outcome = np.mean(y_pred[privileged]) if np.sum(privileged) > 0 else np.nan
    unprivileged_outcome = np.mean(y_pred[unprivileged]) if np.sum(unprivileged) > 0 else np.nan
    if privileged_outcome == 0:
        return np.nan  
    return unprivileged_outcome / privileged_outcome

def equal_opportunity_difference(y_true, y_pred, protected_attribute):
    privileged = protected_attribute == 1
    unprivileged = protected_attribute == 0
    true_positive_rate_privileged = np.sum((y_true[privileged] == 1) & (y_pred[privileged] == 1)) / np.sum(y_true[privileged] == 1)
    true_positive_rate_unprivileged = np.sum((y_true[unprivileged] == 1) & (y_pred[unprivileged] == 1)) / np.sum(y_true[unprivileged] == 1)
    return true_positive_rate_unprivileged - true_positive_rate_privileged

def average_odds_difference(y_true, y_pred, protected_attribute):
    privileged = protected_attribute == 1
    unprivileged = protected_attribute == 0
    tpr_privileged = np.sum((y_true[privileged] == 1) & (y_pred[privileged] == 1)) / np.sum(y_true[privileged] == 1)
    tpr_unprivileged = np.sum((y_true[unprivileged] == 1) & (y_pred[unprivileged] == 1)) / np.sum(y_true[unprivileged] == 1)
    fpr_privileged = np.sum((y_true[privileged] == 0) & (y_pred[privileged] == 1)) / np.sum(y_true[privileged] == 0)
    fpr_unprivileged = np.sum((y_true[unprivileged] == 0) & (y_pred[unprivileged] == 1)) / np.sum(y_true[unprivileged] == 0)
    average_odds_privileged = (tpr_privileged + fpr_privileged) / 2
    average_odds_unprivileged = (tpr_unprivileged + fpr_unprivileged) / 2
    return average_odds_unprivileged - average_odds_privileged

In [8]:
# Adversary model definition
def build_adversary_model(input_shape):
    adversary_input = layers.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(adversary_input)
    x = layers.Dense(32, activation='relu')(x)
    adversary_output = layers.Dense(1, activation='sigmoid')(x)
    adversary_model = models.Model(inputs=adversary_input, outputs=adversary_output)
    adversary_model.compile(optimizer='adam', loss='binary_crossentropy')
    return adversary_model

# Load and preprocess the data
X_train, X_test, y_train, y_test, protected_train, protected_test = load_german()

# Standardize the features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Directory paths
input_directory = './german/german_keras'
output_directory = './german/german_debiased_onnx'

# Ensure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

metrics_filename = './model_metrics/german_model_metrics.csv'

# Iterate over all .keras files in the input directory to convert to ONNX file
for file in os.listdir(input_directory):
    if file.endswith('.keras'):
        # Full path to the current model file
        input_path = os.path.join(input_directory, file)
        output_path = os.path.join(output_directory, file.replace('.keras', '.onnx'))

        try:
            # Load the model
            print(f"Loading model from {input_path}")
            classifier_model = load_model(input_path)

            # Ensure the model is compiled with the correct optimizer and metrics
            classifier_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            # Print metrics for plain model
            y_test_pred_plain = classifier_model.predict(X_test).argmax(axis=1)
            y_test_true = y_test.argmax(axis=1)

            plain_classification_accuracy = classification_accuracy(y_test_true, y_test_pred_plain)
            plain_balanced_accuracy = balanced_accuracy(y_test_true, y_test_pred_plain)
            plain_disparate_impact = disparate_impact(y_test_true, y_test_pred_plain, protected_test)
            plain_equal_opportunity_difference = equal_opportunity_difference(y_test_true, y_test_pred_plain, protected_test)
            plain_average_odds_difference = average_odds_difference(y_test_true, y_test_pred_plain, protected_test)

            save_metrics_to_csv(metrics_filename, file, 'Plain Model', plain_classification_accuracy, plain_balanced_accuracy, plain_disparate_impact, plain_equal_opportunity_difference, plain_average_odds_difference)
            
            # Build and compile the adversary model
            adversary_model = build_adversary_model(classifier_model.output_shape[1:])

            # Training parameters
            num_epochs = 50
            batch_size = 128
            learning_rate = 0.001
            adversary_loss_weight = 0.7

            # Optimizers
            classifier_optimizer = tf.keras.optimizers.Adam(learning_rate)
            adversary_optimizer = tf.keras.optimizers.Adam(learning_rate)

            # Loss functions
            classification_loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
            adversary_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

            # Training loop
            for epoch in range(num_epochs):
                # Shuffle the training data
                indices = np.arange(X_train.shape[0])
                np.random.shuffle(indices)
                
                # Mini-batch training
                for start in range(0, X_train.shape[0], batch_size):
                    end = min(start + batch_size, X_train.shape[0])
                    batch_indices = indices[start:end]
                    
                    X_batch = X_train[batch_indices]
                    y_batch = y_train[batch_indices]
                    protected_batch = protected_train[batch_indices].reshape(-1, 1)
                    
                    with tf.GradientTape() as classifier_tape, tf.GradientTape() as adversary_tape:
                        # Forward pass through the classifier
                        classifier_predictions = classifier_model(X_batch, training=True)
                        
                        # Forward pass through the adversary
                        adversary_predictions = adversary_model(classifier_predictions, training=True)
                        
                        # Compute losses
                        classification_loss = classification_loss_fn(y_batch, classifier_predictions)
                        adversary_loss = adversary_loss_fn(protected_batch, adversary_predictions)
                        total_loss = classification_loss - adversary_loss_weight * adversary_loss
                    
                    # Compute gradients and update classifier weights
                    classifier_gradients = classifier_tape.gradient(total_loss, classifier_model.trainable_variables)
                    classifier_optimizer.apply_gradients(zip(classifier_gradients, classifier_model.trainable_variables))
                    
                    with tf.GradientTape() as adversary_tape:
                        # Forward pass through the classifier
                        classifier_predictions = classifier_model(X_batch, training=True)
                        
                        # Forward pass through the adversary
                        adversary_predictions = adversary_model(classifier_predictions, training=True)
                        
                        # Compute adversary loss
                        adversary_loss = adversary_loss_fn(protected_batch, adversary_predictions)
                    
                    # Compute gradients and update adversary weights
                    adversary_gradients = adversary_tape.gradient(adversary_loss, adversary_model.trainable_variables)
                    adversary_optimizer.apply_gradients(zip(adversary_gradients, adversary_model.trainable_variables))
                
                print(f"Epoch {epoch + 1}/{num_epochs}, Classification Loss: {classification_loss.numpy()}, Adversary Loss: {adversary_loss.numpy()}")
            
            # Predictions for debiased model
            y_test_pred_debiased = classifier_model.predict(X_test).argmax(axis=1)

            debiased_classification_accuracy = classification_accuracy(y_test_true, y_test_pred_debiased)
            debiased_balanced_accuracy = balanced_accuracy(y_test_true, y_test_pred_debiased)
            debiased_disparate_impact = disparate_impact(y_test_true, y_test_pred_debiased, protected_test)
            debiased_equal_opportunity_difference = equal_opportunity_difference(y_test_true, y_test_pred_debiased, protected_test)
            debiased_average_odds_difference = average_odds_difference(y_test_true, y_test_pred_debiased, protected_test)

            save_metrics_to_csv(metrics_filename, file, 'Debiased Model', debiased_classification_accuracy, debiased_balanced_accuracy, debiased_disparate_impact, debiased_equal_opportunity_difference, debiased_average_odds_difference)
            
            # Save the debiased model as ONNX
            input_shape = (20,)  # Adjust the input shape based on your model's expected input
            save_model_onnx(classifier_model, input_shape, output_path)

        except Exception as e:
            print(f"Failed to convert {file}. Error: {e}")

Loading model from ./german/german_keras/GC-1.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Epoch 1/50, Classification Loss: 0.4777572453022003, Adversary Loss: 0.693507194519043
Epoch 2/50, Classification Loss: 0.4861471951007843, Adversary Loss: 0.6141618490219116
Epoch 3/50, Classification Loss: 0.5050150752067566, Adversary Loss: 0.5744163990020752
Epoch 4/50, Classification Loss: 0.4967416822910309, Adversary Loss: 0.5904921889305115
Epoch 5/50, Classification Loss: 0.386519193649292, Adversary Loss: 0.6527164578437805
Epoch 6/50, Classification Loss: 0.43326056003570557, Adversary Loss: 0.5845215320587158
Epoch 7/50, Classification Loss: 0.48554661870002747, Adversary Loss: 0.642375648021698
Epoch 8/50, Classification Loss: 0.4750366806983948, Adversary Loss: 0.6102771162986755
Epoch 9/50, Classification Loss: 0.5047731399536133, Adversary Loss: 0.6226220726966858
Epoch 10/50, Classification Loss: 0.39498353004455566, Adversary Loss: 0.6417691707611084
Epoch 11/50, Classification L

2024-07-10 21:42:20.562575: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:42:20.562733: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:42:20.591673: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:42:20.591837: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


Epoch 1/50, Classification Loss: 0.41636988520622253, Adversary Loss: 0.6667576432228088
Epoch 2/50, Classification Loss: 0.3881838023662567, Adversary Loss: 0.6283025145530701
Epoch 3/50, Classification Loss: 0.4532621502876282, Adversary Loss: 0.604989230632782
Epoch 4/50, Classification Loss: 0.44181907176971436, Adversary Loss: 0.685588538646698
Epoch 5/50, Classification Loss: 0.39300113916397095, Adversary Loss: 0.5952778458595276
Epoch 6/50, Classification Loss: 0.5146796703338623, Adversary Loss: 0.6283681988716125
Epoch 7/50, Classification Loss: 0.4359915554523468, Adversary Loss: 0.6207285523414612
Epoch 8/50, Classification Loss: 0.4620969295501709, Adversary Loss: 0.6686885356903076
Epoch 9/50, Classification Loss: 0.35576242208480835, Adversary Loss: 0.6183834671974182
Epoch 10/50, Classification Loss: 0.4114015996456146, Adversary Loss: 0.5780426859855652
Epoch 11/50, Classification Loss: 0.4176478981971741, Adversary Loss: 0.6144431829452515
Epoch 12/50, Classification 

2024-07-10 21:42:36.623799: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:42:36.623903: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:42:36.639931: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:42:36.640016: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


Epoch 1/50, Classification Loss: 0.5753607749938965, Adversary Loss: 0.6786452531814575
Epoch 2/50, Classification Loss: 0.595988929271698, Adversary Loss: 0.6672905683517456
Epoch 3/50, Classification Loss: 0.4906458854675293, Adversary Loss: 0.648726224899292
Epoch 4/50, Classification Loss: 0.5723000764846802, Adversary Loss: 0.6367440819740295
Epoch 5/50, Classification Loss: 0.5836803913116455, Adversary Loss: 0.6776368618011475
Epoch 6/50, Classification Loss: 0.5085002183914185, Adversary Loss: 0.6295909881591797
Epoch 7/50, Classification Loss: 0.5674439668655396, Adversary Loss: 0.6253600716590881
Epoch 8/50, Classification Loss: 0.565727174282074, Adversary Loss: 0.6324254870414734
Epoch 9/50, Classification Loss: 0.49937132000923157, Adversary Loss: 0.5367003679275513
Epoch 10/50, Classification Loss: 0.5387239456176758, Adversary Loss: 0.6260751485824585
Epoch 11/50, Classification Loss: 0.5506872534751892, Adversary Loss: 0.611318051815033
Epoch 12/50, Classification Loss:

2024-07-10 21:42:52.340111: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:42:52.340195: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:42:52.356494: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:42:52.356581: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


Epoch 1/50, Classification Loss: 0.5646419525146484, Adversary Loss: 0.6427075266838074
Epoch 2/50, Classification Loss: 0.6654554009437561, Adversary Loss: 0.6859434843063354
Epoch 3/50, Classification Loss: 0.5745946168899536, Adversary Loss: 0.6129531264305115
Epoch 4/50, Classification Loss: 0.5645034313201904, Adversary Loss: 0.6247860193252563
Epoch 5/50, Classification Loss: 0.6149442195892334, Adversary Loss: 0.5949403643608093
Epoch 6/50, Classification Loss: 0.6250980496406555, Adversary Loss: 0.6255003809928894
Epoch 7/50, Classification Loss: 0.6149519681930542, Adversary Loss: 0.64559006690979
Epoch 8/50, Classification Loss: 0.6455926895141602, Adversary Loss: 0.6755399107933044
Epoch 9/50, Classification Loss: 0.512831449508667, Adversary Loss: 0.6054505109786987
Epoch 10/50, Classification Loss: 0.5842449069023132, Adversary Loss: 0.6631074547767639
Epoch 11/50, Classification Loss: 0.7178929448127747, Adversary Loss: 0.6058240532875061
Epoch 12/50, Classification Loss:

2024-07-10 21:43:10.153305: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:43:10.153411: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:43:10.173904: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:43:10.173987: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Epoch 1/50, Classification Loss: 0.43383508920669556, Adversary Loss: 0.6606075167655945
Epoch 2/50, Classification Loss: 0.3096054196357727, Adversary Loss: 0.6543052196502686
Epoch 3/50, Classification Loss: 0.38001078367233276, Adversary Loss: 0.6279376149177551
Epoch 4/50, Classification Loss: 0.4212137460708618, Adversary Loss: 0.5857487916946411
Epoch 5/50, Classification Loss: 0.3167945444583893, Adversary Loss: 0.553225040435791
Epoch 6/50, Classification Loss: 0.3123852014541626, Adversary Loss: 0.6062723398208618
Epoch 7/50, Classification Loss: 0.2863142490386963, Adversary Loss: 0.6174367070198059
Epoch 8/50, Classification Loss: 0.32230493426322937, Adversary Loss: 0.74906986951828
Epoch 9/50, Classification Loss: 0.38316476345062256, Adversary Loss: 0.5874425768852234
Epoch 10/50, Classification Loss: 0.3167160749435425, Adversary Loss: 0.6269955039024353
Epoch 11/50, Classification Loss: 0.3110535144805908, Adversary Loss: 0.56981843

2024-07-10 21:43:35.058670: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:43:35.058754: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-10 21:43:35.091426: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-10 21:43:35.091558: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
